# Simple Bayesian Network example

Let's consider this simple network:

```mermaid

graph TD
    Q((Q))
    Y((Y))

    Q-->Y
```

Let's define the following probability distributions.

$P(Q)$ is described by a two-label conditional probability table (CPT):

| q | P(Q = q) |
|---|----------|
| 0 | 0.4      |
| 1 | 0.6      |

$P(Y|Q)$ is described by a CPT where $Y$ takes on three possible values:

| y | P(Y = y &#124; Q = 0) | P(Y = y &#124; Q = 1) |
|---|----------| ------------------------ |
| 0 | 0.1      | 0.5                      |
| 1 | 0.6      | 0.1                      |
| 2 | 0.3      | 0.4                      |

## Inference

### Naive implementation

Now let's perform inference on the hidden variable $Q$ for each of the three possible observations.

The posterior distribution is calculated as:

$$

P(Q|Y) = \frac{P(Y|Q)P(Q)}{P(Y)} = \frac{P(Y|Q)P(Q)}{\sum\limits_{q}{P(Y|Q)P(Q)}}

$$

In [37]:
import numpy as np

# Define CPTs
P_Q = np.array([0.4, 0.6])
P_YxQ = np.array([[0.1, 0.6, 0.3], [0.5, 0.1, 0.4]])

# Calculate unnormalized
P_QxY = (P_YxQ * P_Q[:, None]).T

# Normalize
P_Y = P_QxY.sum(axis=1)
P_QxY /= P_Y[:, None]

# Print
P_QxY


array([[0.11764706, 0.88235294],
       [0.8       , 0.2       ],
       [0.33333333, 0.66666667]])

### Sum-product algorithm

Now let's perform inference using the sum-product algorithm.

The corresponding factor graph is:

```mermaid

graph BT
    classDef evidence visibility:hidden
    classDef variableNode padding:15px
    
    input([" "]):::evidence    
    subgraph group_Y[" "]
        Y(($$ Y $$)):::variableNode
        f2[$$ f_2 $$]
    end
    subgraph group_Q[" "]
        Q(("$$ Q $$")):::variableNode
        f1["$$ f_1 $$"]
    end

    f1--"$$ b_1 $$"-->Q
    Q--"$$ a_1 $$"--> f1
    Q--"$$ a_2 $$"-->f2
    f2--"$$ b_2 $$"-->Q
    f2--"$$ b_3 $$"-->Y
    Y--"$$ a_3 $$"-->f2
    
    input-.->|$$ d_3 $$| Y
    Y ~~~ input
    
```

And message definitions:

$$

\begin{align}

d_3(y) & = 
    P(\hat{Y}|Y)
    \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad 
    & \begin{cases}
        1 & \text{if } y = \hat{y} \\
        0 & \text{if } y \ne \hat{y} \\
    \end{cases} \\
a_3(y) & = 
    d_3(y)
    & P(\hat{Y}|Y) \\
b_3(y) & =
    \sum_{q}{f_2(q, y)a_2(q)} 
    = \sum_{q}{P(Y|Q)P(Q)}
    & P(Y) \\
b_2(q) & =
    \sum_{q}{f_2(q, y)a_3(q)} 
    = \sum_{q}{P(Y|Q)P(\hat{Y}|Y)}
    & P(\hat{Y}|Q) \\
a_2(q) & = 
    b_1(q)
    & P(Q) \\
a_1(q) & = 
    b_2(q)
    & P(\hat{Y}|Q) \\
b_1(q) & = 
    f_1(q)
    & P(Q) \\
        

\end{align}

$$


In [38]:
P_QxY = np.zeros((3, 2))

# For each possible input value
for Y_hat in [0, 1, 2]:
    # Messages upwards
    d3 = (np.arange(3) == Y_hat).astype(float)
    a3 = d3
    b2 = (P_YxQ * a3[None, :]).sum(axis=1)
    a1 = b2

    # Messages downwards
    b1 = P_Q
    a2 = b1
    b3 = (P_YxQ * a2[:, None]).sum(axis=0)
    
    # Calculate P_Q
    P_QxY[Y_hat, :] = a1 * b1 / (a1 * b1).sum()
    
P_QxY
    

array([[0.11764706, 0.88235294],
       [0.8       , 0.2       ],
       [0.33333333, 0.66666667]])

These values match with the ones calculated naively, showing the correctness of the algorithm.